In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor

In [2]:
k=1
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i_fnsd.cmd' % k)
solverF.init_grid()
solverF.init_flux_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFNSR%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, transient=True)
solverS.output_file = 'solid_outputs/resFNSR%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_robin_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
abTempF = InterfaceData(size=inodes, value=300.0)
abTempF2S = InterfaceData(size=inodes, value=300.0)
FluxS = InterfaceData(size=inodes, value=0.0)
FluxF = InterfaceData(size=inodes, value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=1e-6)

# maximum pc steps allowed
max_pc_steps = 200

# since this problem has no mesh motion and both kappa are material properties
# so the heat transfer coefficients don't vary in time
hF2S = solverF.get_heat_transfer_coeff(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)[f2s]

In [7]:
# coupling timing information, use fixed time step for now
Tfinal = 1.0
N = 100
dt = Tfinal/N
t = 0.0
avg_pc_iterations = 0
flog = open('FNSR%i.log' % k, mode='w')
flog.write('Fluid Neumann with solid Robin setting, study case: %i\n' % k)
flog.close()
flog = open('FNSR%i.log' % k, mode='a')

In [8]:
# begin to couple
for step in range(N):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    while True:
        pc_counts = pc_counts + 1
        
        # back up previous interface value
        FluxF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid interface flux
        abTempF.curr[:] = solverF.get_ambient_temperature(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        abTempF2S.curr[:] = abTempF.curr[f2s]
        # update solid flux interface
        solverS.update_robin_interface(
            tag=interface_tag,
            Ta=abTempF2S.curr,
            h_coeff=hF2S
        )
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface temperature
        FluxS.curr[:] = solverS.get_interface_flux(tag=interface_tag)
        
        # mapping
        FluxF.curr[:] = FluxS.curr[s2f]
        
        # update residual
        FluxF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(FluxF)
        if is_conv or pc_counts >= max_pc_steps:
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            under_relax.determine_omega(FluxF)
            under_relax.update_solution(FluxF)
            solverF.put_heat_flux_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=FluxF.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore()  

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    solverS.write()
    fluid_ofile.write(solverF, t)
    avg_pc_iterations = avg_pc_iterations + pc_counts
flog.close()

step=0, pc_iterations=8.
step=1, pc_iterations=8.
step=2, pc_iterations=7.
step=3, pc_iterations=7.
step=4, pc_iterations=7.
step=5, pc_iterations=7.
step=6, pc_iterations=7.
step=7, pc_iterations=7.
step=8, pc_iterations=7.
step=9, pc_iterations=7.
step=10, pc_iterations=7.
step=11, pc_iterations=7.
step=12, pc_iterations=7.
step=13, pc_iterations=7.
step=14, pc_iterations=7.
step=15, pc_iterations=7.
step=16, pc_iterations=7.
step=17, pc_iterations=7.
step=18, pc_iterations=7.
step=19, pc_iterations=7.
step=20, pc_iterations=7.
step=21, pc_iterations=7.
step=22, pc_iterations=7.
step=23, pc_iterations=7.
step=24, pc_iterations=7.
step=25, pc_iterations=7.
step=26, pc_iterations=7.
step=27, pc_iterations=6.
step=28, pc_iterations=7.
step=29, pc_iterations=6.
step=30, pc_iterations=7.
step=31, pc_iterations=6.
step=32, pc_iterations=7.
step=33, pc_iterations=6.
step=34, pc_iterations=6.
step=35, pc_iterations=6.
step=36, pc_iterations=6.
step=37, pc_iterations=6.
step=38, pc_iterations

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat('numeric_res/kFNSR%i.mat' % k, {'theta': theta, 'avg_pc_its': avg_pc_iterations/N})

[0.25667754 0.27960064 0.29493925 0.3057843  0.31508994 0.32341581
 0.33075586 0.33736323 0.34349094 0.34893211 0.35396691 0.35855873
 0.36287297 0.36681728 0.37048794 0.37385884 0.37702448 0.37995106
 0.38265172 0.3851867  0.38752415 0.38970258 0.39172926 0.39360043
 0.39534723 0.39695541 0.39846514 0.39984975 0.40111919 0.40230681
 0.40341054 0.40442072 0.40536088 0.40622114 0.40701821 0.40774638
 0.40841902 0.4090299  0.40960022 0.41011749 0.41058773 0.41101047
 0.411397   0.41174828 0.41206496 0.41234832 0.41260162 0.41282475
 0.41301751 0.41318814 0.41333053 0.41344988 0.41354446 0.41361649
 0.41366515 0.41369515 0.41369958 0.41368253 0.41364379 0.4135804
 0.41349135 0.41338291 0.41324574 0.41308249 0.41288809 0.41266769
 0.41240962 0.41212094 0.41178941 0.41142083 0.41101479 0.41055427
 0.4100374  0.4094766  0.40884634 0.40815116 0.4073751  0.40652529
 0.40557486 0.40453532 0.40337511 0.40209885 0.40069356 0.39914478
 0.39742356 0.39550025 0.39340577 0.39104993 0.38841467 0.38550